# Week 13: Testing, Validation, & Ethics

### Topics Covered

- Hardware-in-the-Loop (HIL) and Software-in-the-Loop (SIL) simulation; Edge Cases and Scenario Testing; AV Ethics (Liability, The Trolley Problem)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, FancyBboxPatch, Polygon, Wedge
from matplotlib.collections import PatchCollection
from dataclasses import dataclass
from typing import List, Tuple
import itertools

# Set random seed for reproducibility
np.random.seed(42)

## 1. Testing and Validation Fundamentals

Testing and validation are critical for ensuring autonomous vehicle safety and reliability.

### Verification vs. Validation

**Verification**: "Are we building the product right?"
- Confirms implementation matches specification
- Methods: Unit tests, integration tests, code review
- Question: Does the code do what we designed?

**Validation**: "Are we building the right product?"
- Confirms system meets user needs and safety goals
- Methods: Field testing, user studies, scenario testing
- Question: Does the system solve the actual problem safely?

### Testing Pyramid for Autonomous Vehicles

```
                 /\
                /  \  Field Testing (Hours-Days)
               /    \
              /------\
             / System \ Integration Testing (Minutes-Hours)
            /  Tests   \
           /------------\
          /  Component   \ Unit Tests (Milliseconds-Seconds)
         /     Tests      \
        /------------------\
       /   Simulation (XiL)  \ Simulation Testing (Microseconds-Milliseconds)
      /----------------------\
```

**Bottom to Top:**
1. **Simulation (XiL)**: Model/Software/Hardware-in-the-Loop
2. **Unit Tests**: Individual functions and modules
3. **Component Tests**: Subsystems (perception, planning, control)
4. **Integration Tests**: Full stack on test track
5. **Field Testing**: Real-world driving

### Types of Testing

**1. Functional Testing**
- Does each feature work as specified?
- Example: Lane keeping maintains center of lane ± 0.3m

**2. Performance Testing**
- How well does it work?
- Example: Object detection achieves 95% recall at 100m

**3. Safety Testing**
- What happens when things go wrong?
- Example: System safely stops when sensor fails

**4. Robustness Testing**
- How does it handle edge cases?
- Example: Performance in fog, rain, direct sunlight

**5. Regression Testing**
- Did new changes break existing functionality?
- Automated test suites run on every code change

---

## 2. Simulation-Based Testing

Simulation enables faster, cheaper, and safer testing than real-world driving.

### X-in-the-Loop (XiL) Testing

**Model-in-the-Loop (MiL)**
- Pure software simulation
- Algorithms run on development PC
- Fastest iteration, lowest fidelity
- Use: Algorithm development

**Software-in-the-Loop (SiL)**
- Production code compiled for PC
- Tests actual software implementation
- Use: Software verification

**Hardware-in-the-Loop (HIL)**
- Real ECU hardware, simulated environment
- Sensors receive synthetic signals
- Highest fidelity before vehicle
- Use: Integration testing

**Vehicle-in-the-Loop (ViL)**
- Real vehicle on dynamometer
- Virtual environment projected
- Use: Final validation

### Simulation Fidelity Levels

**Level 1: Kinematic**
- Simple physics (no tire forces)
- Fast, used for path planning

**Level 2: Dynamic**
- Tire models, suspension
- Used for control testing

**Level 3: High-Fidelity**
- Detailed sensor models (ray-tracing for lidar)
- Weather, lighting effects
- Used for perception testing

**Level 4: Photo-Realistic**
- Game-engine graphics
- Used for camera algorithm validation

### Simulation Advantages & Limitations

**Advantages:**
- **Safety**: Test dangerous scenarios without risk
- **Speed**: Run 10-100x faster than real-time
- **Coverage**: Test rare events (e.g., tire blowout)
- **Reproducibility**: Exact replay for debugging
- **Cost**: No physical vehicle/track needed

**Limitations:**
- **Sim2Real Gap**: Simulated sensors ≠ real sensors
- **Unknown Unknowns**: Can't simulate what you haven't modeled
- **Validation**: How to validate the simulator itself?
- **Complexity**: High-fidelity sims are expensive to build

In [ ]:
# Implementation: Simple SIL Simulation Framework

@dataclass
class VehicleState:
    """Vehicle state representation"""
    x: float = 0.0
    y: float = 0.0
    heading: float = 0.0
    velocity: float = 0.0

@dataclass
class SensorReading:
    """Simulated sensor reading"""
    distance: float
    angle: float
    object_type: str

class VehicleSimulator:
    """Software-in-the-Loop vehicle simulator"""
    
    def __init__(self, dt=0.1):
        self.dt = dt
        self.state = VehicleState()
        self.obstacles = []
    
    def add_obstacle(self, x, y, radius=1.0, obj_type='vehicle'):
        """Add obstacle to environment"""
        self.obstacles.append({'x': x, 'y': y, 'radius': radius, 'type': obj_type})
    
    def simulate_sensor(self, sensor_range=50.0, sensor_fov=np.pi):
        """Simulate range sensor (lidar/radar)"""
        readings = []
        
        for obs in self.obstacles:
            # Calculate relative position
            dx = obs['x'] - self.state.x
            dy = obs['y'] - self.state.y
            distance = np.sqrt(dx**2 + dy**2)
            angle = np.arctan2(dy, dx) - self.state.heading
            
            # Normalize angle to [-pi, pi]
            angle = (angle + np.pi) % (2 * np.pi) - np.pi
            
            # Check if in sensor FOV and range
            if distance <= sensor_range and abs(angle) <= sensor_fov / 2:
                # Add measurement noise
                noisy_distance = distance + np.random.normal(0, 0.1)
                noisy_angle = angle + np.random.normal(0, 0.01)
                
                readings.append(SensorReading(
                    distance=max(0, noisy_distance),
                    angle=noisy_angle,
                    object_type=obs['type']
                ))
        
        return readings
    
    def apply_control(self, acceleration, steering_angle):
        """Apply control inputs and update state"""
        # Simple kinematic bicycle model
        L = 2.5  # Wheelbase
        
        # Update velocity
        self.state.velocity += acceleration * self.dt
        self.state.velocity = np.clip(self.state.velocity, 0, 30)  # 0-30 m/s
        
        # Update heading
        self.state.heading += (self.state.velocity / L) * np.tan(steering_angle) * self.dt
        
        # Update position
        self.state.x += self.state.velocity * np.cos(self.state.heading) * self.dt
        self.state.y += self.state.velocity * np.sin(self.state.heading) * self.dt
        
        return self.state


class SimpleACC:
    """Adaptive Cruise Control algorithm under test"""
    
    def __init__(self, target_speed=20.0, safe_distance=10.0):
        self.target_speed = target_speed
        self.safe_distance = safe_distance
        self.Kp = 0.5  # Proportional gain
    
    def compute_control(self, current_velocity, sensor_readings):
        """Compute acceleration command based on sensor readings"""
        # Find closest obstacle ahead
        closest_distance = float('inf')
        
        for reading in sensor_readings:
            # Only consider objects roughly ahead (within ±30 degrees)
            if abs(reading.angle) < np.deg2rad(30):
                if reading.distance < closest_distance:
                    closest_distance = reading.distance
        
        # If obstacle detected, maintain safe distance
        if closest_distance < self.safe_distance * 2:
            # Slow down
            error = (closest_distance - self.safe_distance) * 0.3
            acceleration = self.Kp * error
        else:
            # Speed up to target speed
            error = self.target_speed - current_velocity
            acceleration = self.Kp * error
        
        # Clip acceleration
        return np.clip(acceleration, -3.0, 2.0)


# Demonstration: SIL Testing of ACC
def demonstrate_sil_testing():
    print("=== Software-in-the-Loop (SIL) Testing ===\n")
    print("Testing Adaptive Cruise Control Algorithm\n")
    
    # Create simulator
    sim = VehicleSimulator(dt=0.1)
    
    # Create ACC controller
    acc = SimpleACC(target_speed=20.0, safe_distance=10.0)
    
    # Test scenario: Vehicle ahead decelerates
    print("Scenario: Following vehicle that decelerates\n")
    
    # Add leading vehicle
    lead_vehicle_x = 30.0
    lead_vehicle_speed = 20.0
    
    # Simulation loop
    time_steps = 200
    history = {
        'time': [],
        'ego_x': [],
        'ego_v': [],
        'lead_x': [],
        'distance': [],
        'acceleration': []
    }
    
    for t in range(time_steps):
        time = t * sim.dt
        
        # Update lead vehicle position (decelerate after t=5s)
        if time > 5.0:
            lead_vehicle_speed = max(10.0, lead_vehicle_speed - 0.05)
        lead_vehicle_x += lead_vehicle_speed * sim.dt
        
        # Clear and re-add obstacles
        sim.obstacles = []
        sim.add_obstacle(lead_vehicle_x, 0, radius=2.0, obj_type='vehicle')
        
        # Simulate sensors
        readings = sim.simulate_sensor()
        
        # Compute control
        accel_cmd = acc.compute_control(sim.state.velocity, readings)
        
        # Apply control (no steering for this test)
        sim.apply_control(accel_cmd, 0.0)
        
        # Record history
        distance = lead_vehicle_x - sim.state.x if len(readings) > 0 else float('inf')
        history['time'].append(time)
        history['ego_x'].append(sim.state.x)
        history['ego_v'].append(sim.state.velocity)
        history['lead_x'].append(lead_vehicle_x)
        history['distance'].append(distance)
        history['acceleration'].append(accel_cmd)
    
    # Analyze results
    min_distance = min(history['distance'])
    collision = min_distance < 2.0  # Vehicle length ~5m, consider collision if < 2m gap
    
    print(f"Test Results:")
    print(f"  Duration: {time_steps * sim.dt:.1f} seconds")
    print(f"  Minimum distance: {min_distance:.2f} m")
    print(f"  Collision: {'YES - FAIL' if collision else 'NO - PASS'}")
    print(f"  Final ego velocity: {history['ego_v'][-1]:.2f} m/s")
    print(f"  Final lead velocity: {lead_vehicle_speed:.2f} m/s")
    
    # Visualize
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Position over time
    ax1 = axes[0, 0]
    ax1.plot(history['time'], history['ego_x'], 'b-', linewidth=2, label='Ego Vehicle')
    ax1.plot(history['time'], history['lead_x'], 'r--', linewidth=2, label='Lead Vehicle')
    ax1.set_xlabel('Time (s)', fontsize=11)
    ax1.set_ylabel('Position (m)', fontsize=11)
    ax1.set_title('Vehicle Positions', fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Velocity over time
    ax2 = axes[0, 1]
    ax2.plot(history['time'], history['ego_v'], 'b-', linewidth=2, label='Ego Velocity')
    ax2.axhline(y=acc.target_speed, color='g', linestyle=':', linewidth=1.5, label='Target Speed')
    ax2.set_xlabel('Time (s)', fontsize=11)
    ax2.set_ylabel('Velocity (m/s)', fontsize=11)
    ax2.set_title('Ego Vehicle Velocity', fontsize=13, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Distance to lead vehicle
    ax3 = axes[1, 0]
    ax3.plot(history['time'], history['distance'], 'g-', linewidth=2)
    ax3.axhline(y=acc.safe_distance, color='r', linestyle='--', linewidth=1.5, label='Safe Distance')
    ax3.axhline(y=2.0, color='darkred', linestyle='--', linewidth=2, label='Collision Threshold')
    ax3.set_xlabel('Time (s)', fontsize=11)
    ax3.set_ylabel('Distance (m)', fontsize=11)
    ax3.set_title('Following Distance', fontsize=13, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Acceleration command
    ax4 = axes[1, 1]
    ax4.plot(history['time'], history['acceleration'], 'orange', linewidth=2)
    ax4.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax4.set_xlabel('Time (s)', fontsize=11)
    ax4.set_ylabel('Acceleration (m/s²)', fontsize=11)
    ax4.set_title('Control Command', fontsize=13, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Test verdict
    print(f"\n{'='*50}")
    if not collision and min_distance >= acc.safe_distance * 0.8:
        print("✓ TEST PASSED: ACC maintains safe distance")
    else:
        print("✗ TEST FAILED: Safety requirements violated")
    print(f"{'='*50}")

demonstrate_sil_testing()

---

## 3. Scenario-Based Testing and Edge Cases

Scenario-based testing focuses on specific situations rather than random driving miles.

### What is a Scenario?

A **scenario** is a specific driving situation with defined:
- **Actors**: Ego vehicle, other vehicles, pedestrians
- **Environment**: Road type, weather, lighting
- **Events**: Actions and their timing

**Example Scenario:**
```
Scenario: Pedestrian Crosses at Unmarked Location
- Road: Urban street, 30 mph limit
- Weather: Clear, daytime
- Event: Pedestrian steps off curb 20m ahead
- Expected: Vehicle brakes, avoids collision
```

### Scenario Taxonomy

**Functional Scenarios** (Abstract)
- High-level description
- Example: "Vehicle performs lane change"

**Logical Scenarios** (Parameterized)
- Specific parameter ranges
- Example: "Lane change at 50-70 mph, gap 20-40m"

**Concrete Scenarios** (Instantiated)
- Exact parameters
- Example: "Lane change at 65 mph, 30m gap, wet road"

### Edge Cases and Corner Cases

**Edge Case**: Extreme but possible situation
- Example: Glare from setting sun directly ahead
- Example: Truck with reflective surface causing sensor interference

**Corner Case**: Combination of multiple edge cases
- Example: Glare + wet road + pedestrian in shadow
- Much rarer but higher risk

### Scenario Generation Approaches

**1. Expert-Defined**
- Safety engineers list dangerous scenarios
- Based on accident data, near-misses
- Example: NHTSA pre-crash scenarios

**2. Data-Driven**
- Extract scenarios from real-world driving logs
- Cluster similar situations
- Example: Waymo extracts scenarios from 20M miles

**3. Adversarial**
- Systematically find failures
- Optimization finds worst-case parameters
- Example: Maximize collision probability

**4. Combinatorial**
- Test all combinations of parameters
- Covering arrays reduce test count
- Example: Pairwise testing of weather × lighting × speed

### Critical Scenarios for AVs

**Cut-In Scenarios:**
- Vehicle suddenly enters ego lane
- Varied: Speed difference, gap distance, driver reaction

**Pedestrian Scenarios:**
- Crossing (marked/unmarked crosswalk)
- Occlusion (behind parked car)
- Distracted (looking at phone)

**Intersection Scenarios:**
- Red light violation by other vehicle
- Unprotected left turn (oncoming traffic)
- Four-way stop coordination

**Sensor Challenge Scenarios:**
- Fog (reduces lidar/camera range)
- Direct sunlight (camera saturation)
- Reflective surfaces (radar false positives)

### Coverage Metrics

**Scenario Coverage**: % of identified scenarios tested

**Parameter Coverage**: % of parameter space explored

**Code Coverage**: % of software lines executed
- Statement coverage
- Branch coverage
- Path coverage (impractical for large systems)

**Requirement Coverage**: % of requirements validated

---

## 4. Ethics and Societal Implications

Autonomous vehicles raise profound ethical questions about decision-making and responsibility.

### The Trolley Problem for AVs

**Classic Trolley Problem:**
A runaway trolley will kill 5 people on the track. You can pull a lever to divert it to another track, killing 1 person instead. Do you pull the lever?

**AV Adaptation:**
An AV's brakes fail. It can either:
- **Option A**: Continue straight, hitting 5 pedestrians
- **Option B**: Swerve, hitting 1 pedestrian on the sidewalk

**Key Differences from Classic Problem:**
1. **Uncertainty**: Sensors have errors, outcomes are probabilistic
2. **Time Pressure**: Decision must be made in milliseconds
3. **Preprogrammed**: Decision logic written before the situation occurs
4. **Legal Liability**: Who is responsible for the programmed decision?

### Ethical Frameworks

**1. Utilitarianism**
- Maximize overall well-being
- "Greatest good for the greatest number"
- AV implication: Minimize total harm (injuries, deaths)
- Problem: May sacrifice individual for collective good

**2. Deontology (Kant)**
- Follow moral rules/duties
- "Never use people as means to an end"
- AV implication: Never intentionally harm anyone
- Problem: May result in worse overall outcomes

**3. Virtue Ethics**
- Act as a virtuous person would
- Consider character and intentions
- AV implication: Design systems that reflect societal values
- Problem: Vague guidance for specific situations

**4. Social Contract Theory**
- Rules people would agree to behind "veil of ignorance"
- AV implication: Policies acceptable to all stakeholders
- Problem: Different stakeholders have conflicting interests

### Moral Machine Experiment

MIT's Moral Machine collected 40 million decisions from millions of people worldwide on AV dilemmas.

**Key Findings:**
1. **Preference for fewer casualties**: Most prefer saving more lives
2. **Preference for children over elderly**: Strong preference to save children
3. **Preference for humans over animals**: Humans valued more
4. **Cultural variation**: Eastern vs. Western cultures differ significantly
5. **Preference for law-abiding**: Pedestrians crossing legally valued more

**Problem**: No global consensus on AV ethics

### Ethical Considerations in AV Design

**1. Impartiality vs. Self-Preference**
- Should AV protect passengers or minimize total harm?
- Survey paradox: People want "ethical" AVs but prefer buying "selfish" AVs

**2. Transparency vs. Complexity**
- Should AV decision-making be explainable?
- Deep learning models are often "black boxes"

**3. Privacy vs. Safety**
- AV cameras record everything
- Who owns the data? How long is it kept?
- Can it be subpoenaed in legal cases?

**4. Accessibility vs. Profitability**
- Will AVs reduce mobility inequality or increase it?
- Who can afford autonomous mobility?

**5. Employment Impact**
- 3.5 million truck drivers in US
- Taxi/Uber drivers
- What happens to displaced workers?

### Liability and Legal Frameworks

**Who is Liable When AV Crashes?**

**Traditional View (human-driven):**
- Driver is liable for negligence
- Sometimes: Vehicle manufacturer (defects)

**AV Scenarios:**
1. **Manufacturing Defect**: Manufacturer liable
2. **Software Bug**: Software developer? OEM? Both?
3. **Sensor Limitation (SOTIF)**: Who knew about limitation?
4. **AI Decision**: How to prove negligence in ML system?
5. **Cybersecurity Attack**: Hacker? Manufacturer for poor security?

**Regulatory Approaches:**

**Prescriptive Regulation:**
- Specific rules (e.g., "must brake within 2 seconds")
- Clear compliance criteria
- Problem: Stifles innovation, can't anticipate all scenarios

**Performance-Based Regulation:**
- Outcomes required (e.g., "must be safer than humans")
- Flexible implementation
- Problem: Hard to measure, verify

**Current Status:**
- No comprehensive federal AV regulations in US (as of 2023)
- Patchwork of state laws
- EU proposed AI Act (includes AVs)
- China has specific AV regulations

### Social Acceptance

**Factors Affecting Public Acceptance:**

1. **Trust**: Reliability must be demonstrated
2. **Control**: Loss of control anxiety
3. **Transparency**: Understanding how decisions are made
4. **Experience**: Exposure increases acceptance
5. **Incidents**: High-profile crashes reduce trust significantly

**Zero-Risk Fallacy:**
- AVs held to impossibly high standard
- Humans cause 1.3M deaths/year globally
- Single AV fatality gets massive media attention
- AVs must be significantly safer, not perfect

---

## Exercises

### Exercise 1: Scenario Test Suite Design

**Objective:** Design a comprehensive test suite for an emergency braking system using scenario-based testing.

**Task:** Create a scenario database covering critical pedestrian crossing situations.

**Instructions:**
- Define at least 10 concrete scenarios varying:
  - Pedestrian speed (slow walk, normal, running)
  - Initial distance (10m, 20m, 30m)
  - Ego vehicle speed (30, 50, 70 km/h)
  - Occlusion (none, partial, full)
  - Lighting (day, dusk, night)
- For each scenario, specify:
  - Expected outcome (stop before ped, collision, near-miss)
  - Pass/fail criteria (e.g., TTC > 1.5s)
  - Priority (P0=critical, P1=important, P2=nice-to-have)
- Organize scenarios using equivalence partitioning
- Calculate total test time assuming 2 min/scenario

**Deliverables:**
- Scenario database (table format)
- Coverage analysis (what parameter combinations are covered?)
- Test execution plan (order, priorities)

### Exercise 2: SIL Testing with Fault Injection

**Objective:** Extend the SIL simulator to test system robustness under sensor failures.

**Task:** Implement fault injection in the `VehicleSimulator` class and test ACC behavior.

**Instructions:**
- Add fault injection capability to `simulate_sensor()`:
  - Sensor dropout (returns no readings)
  - Degraded accuracy (increased noise)
  - Stuck reading (returns last valid reading)
  - Delayed reading (latency simulation)
- Test ACC with each fault type
- Measure:
  - Time to detect fault
  - Safety impact (minimum distance maintained)
  - Recovery behavior
- Plot results comparing normal vs. faulty operation

**Requirements:**
- ACC must detect sensor fault within 0.5 seconds
- ACC must enter safe state (gradual deceleration) upon fault
- No collisions even with sensor failure

### Exercise 3: Ethical Decision Framework

**Objective:** Analyze ethical dilemmas in AV decision-making and propose a decision framework.

**Task:** Create a decision tree for handling unavoidable collision scenarios.

**Instructions:**
1. Define 5 unavoidable collision scenarios:
   - Example: Brakes fail, must choose between hitting pedestrian or swerving into barrier (risking passenger)
   
2. For each scenario, analyze:
   - Utilitarian outcome (minimize total harm)
   - Deontological outcome (follow rules, e.g., "never swerve into pedestrian")
   - Legal considerations
   - Public opinion (based on Moral Machine findings)

3. Propose a decision framework:
   - Hierarchy of factors to consider
   - Weighting of different outcomes
   - Tie-breaking rules

4. Implement a simple decision function:
   ```python
   def choose_action(scenario):
       # Input: Scenario with options and estimated outcomes
       # Output: Chosen action with justification
       pass
   ```

5. Test your framework on all 5 scenarios
6. Compare your framework's decisions with:
   - Human survey responses
   - Other ethical frameworks
   - Current AV manufacturer policies (if known)

**Reflection Questions:**
- Should AVs prioritize passengers or pedestrians?
- Should probabilities matter? (50% chance of hitting 1 vs. 100% chance of hitting 0.3 people)
- How should uncertainty affect decisions?
- Is it ethical to program different behaviors for different markets?

In [ ]:
# Exercise Solutions

# Exercise 1: Scenario Test Suite Design
# TODO: Create scenario database
#
# Example structure:
# import pandas as pd
#
# scenarios = [
#     {
#         'id': 'PED-001',
#         'description': 'Adult pedestrian crosses at marked crosswalk',
#         'ped_speed': 'normal',  # 1.4 m/s
#         'initial_distance': 20,  # meters
#         'ego_speed': 50,  # km/h
#         'occlusion': 'none',
#         'lighting': 'day',
#         'expected_outcome': 'stop',
#         'pass_criteria': 'TTC > 1.5s, no collision',
#         'priority': 'P0'
#     },
#     {
#         'id': 'PED-002',
#         'description': 'Child runs into street from behind parked car',
#         'ped_speed': 'running',  # 3.0 m/s
#         'initial_distance': 10,  # meters
#         'ego_speed': 30,  # km/h
#         'occlusion': 'full',
#         'lighting': 'day',
#         'expected_outcome': 'near-miss or collision',
#         'pass_criteria': 'emergency braking activated, speed < 20 km/h at impact',
#         'priority': 'P0'
#     },
#     # ... add 8 more scenarios
# ]
#
# df = pd.DataFrame(scenarios)
# print(df[['id', 'description', 'priority', 'expected_outcome']])
#
# # Coverage analysis
# print("\nParameter Coverage:")
# print(f"  Pedestrian speeds tested: {df['ped_speed'].nunique()}")
# print(f"  Initial distances tested: {df['initial_distance'].nunique()}")
# print(f"  Ego speeds tested: {df['ego_speed'].nunique()}")
# print(f"  Occlusion types tested: {df['occlusion'].nunique()}")
# print(f"  Lighting conditions tested: {df['lighting'].nunique()}")
#
# # Test execution plan
# p0_scenarios = df[df['priority'] == 'P0']
# print(f"\nTest Execution Plan:")
# print(f"  Total scenarios: {len(df)}")
# print(f"  P0 (critical): {len(p0_scenarios)} - Test first")
# print(f"  Estimated time: {len(df) * 2} minutes")


# Exercise 2: SIL Testing with Fault Injection
# TODO: Extend VehicleSimulator with fault injection
#
# Example structure:
# class FaultInjector:
#     def __init__(self, fault_type='none', fault_start_time=0, fault_duration=float('inf')):
#         self.fault_type = fault_type
#         self.fault_start_time = fault_start_time
#         self.fault_duration = fault_duration
#         self.time = 0
#         self.last_valid_reading = None
#     
#     def update(self, dt):
#         self.time += dt
#     
#     def is_active(self):
#         return (self.time >= self.fault_start_time and 
#                 self.time < self.fault_start_time + self.fault_duration)
#     
#     def apply_fault(self, readings):
#         if not self.is_active():
#             self.last_valid_reading = readings.copy() if readings else None
#             return readings
#         
#         if self.fault_type == 'dropout':
#             return []  # No readings
#         
#         elif self.fault_type == 'degraded':
#             # Increase noise
#             noisy_readings = []
#             for r in readings:
#                 noisy_r = SensorReading(
#                     distance=r.distance + np.random.normal(0, 2.0),  # 20x normal noise
#                     angle=r.angle + np.random.normal(0, 0.1),
#                     object_type=r.object_type
#                 )
#                 noisy_readings.append(noisy_r)
#             return noisy_readings
#         
#         elif self.fault_type == 'stuck':
#             # Return last valid reading
#             return self.last_valid_reading if self.last_valid_reading else []
#         
#         elif self.fault_type == 'delayed':
#             # Return readings from 0.2s ago (would need history buffer)
#             # Simplified: just add latency to distance
#             delayed_readings = []
#             for r in readings:
#                 delayed_r = SensorReading(
#                     distance=r.distance + 2.0,  # Approximate 0.2s delay at 10 m/s
#                     angle=r.angle,
#                     object_type=r.object_type
#                 )
#                 delayed_readings.append(delayed_r)
#             return delayed_readings
#
# # Test each fault type
# fault_types = ['none', 'dropout', 'degraded', 'stuck', 'delayed']
# results = {}
#
# for fault_type in fault_types:
#     sim = VehicleSimulator(dt=0.1)
#     acc = SimpleACC(target_speed=20.0, safe_distance=10.0)
#     fault_injector = FaultInjector(fault_type=fault_type, fault_start_time=5.0, fault_duration=3.0)
#     
#     # Run simulation (similar to demonstrate_sil_testing)
#     # ... simulation loop with fault_injector.apply_fault(readings) ...
#     
#     results[fault_type] = {
#         'min_distance': min_distance,
#         'collision': collision,
#         'fault_detected': True,  # Implement fault detection logic
#         'detection_time': 0.3  # Time to detect fault
#     }
#
# # Compare results
# for fault_type, result in results.items():
#     print(f"{fault_type}: min_dist={result['min_distance']:.2f}m, "
#           f"collision={result['collision']}, "
#           f"detected={result['fault_detected']}")


# Exercise 3: Ethical Decision Framework
# TODO: Implement ethical decision framework
#
# Example structure:
# @dataclass
# class Outcome:
#     action: str
#     passengers_harmed: int
#     pedestrians_harmed: int
#     property_damage: float
#     probability: float = 1.0
#     
#     def expected_harm(self):
#         # Weighted harm calculation
#         return self.probability * (
#             10.0 * self.passengers_harmed +
#             10.0 * self.pedestrians_harmed +
#             0.001 * self.property_damage
#         )
#
# class EthicalDecisionFramework:
#     def __init__(self, framework='utilitarian'):
#         self.framework = framework
#     
#     def choose_action(self, outcomes: List[Outcome]) -> Outcome:
#         if self.framework == 'utilitarian':
#             # Minimize expected harm
#             return min(outcomes, key=lambda o: o.expected_harm())
#         
#         elif self.framework == 'deontological':
#             # Never intentionally harm pedestrians
#             # Prefer actions that don't swerve into pedestrians
#             pedestrian_safe = [o for o in outcomes if o.pedestrians_harmed == 0]
#             if pedestrian_safe:
#                 return min(pedestrian_safe, key=lambda o: o.expected_harm())
#             else:
#                 # All options harm pedestrians, choose least harm
#                 return min(outcomes, key=lambda o: o.pedestrians_harmed)
#         
#         elif self.framework == 'passenger_priority':
#             # Prioritize passengers
#             return min(outcomes, key=lambda o: (o.passengers_harmed, o.expected_harm()))
#
# # Example scenario
# scenario_brake_failure = [
#     Outcome(action='continue_straight', passengers_harmed=0, pedestrians_harmed=5, property_damage=0),
#     Outcome(action='swerve_right', passengers_harmed=2, pedestrians_harmed=0, property_damage=50000),
#     Outcome(action='brake_hard', passengers_harmed=0, pedestrians_harmed=3, property_damage=0, probability=0.7)
# ]
#
# # Test different frameworks
# for framework_name in ['utilitarian', 'deontological', 'passenger_priority']:
#     framework = EthicalDecisionFramework(framework_name)
#     choice = framework.choose_action(scenario_brake_failure)
#     print(f"{framework_name}: chose '{choice.action}'")
#     print(f"  Expected harm: {choice.expected_harm():.2f}")
#     print(f"  Passengers harmed: {choice.passengers_harmed}")
#     print(f"  Pedestrians harmed: {choice.pedestrians_harmed}\n")

---

## References

### Books - Testing & Validation

1. **Ammann, P., & Offutt, J.** (2016). *Introduction to Software Testing* (2nd ed.). Cambridge University Press.
   - Comprehensive software testing fundamentals
   - Coverage criteria, test case generation

2. **Copeland, L.** (2004). *A Practitioner's Guide to Software Test Design*. Artech House.
   - Practical testing techniques
   - Scenario-based testing methods

### Papers - Testing & Validation

3. **Kalra, N., & Paddock, S. M.** (2016). "Driving to safety: How many miles of driving would it take to demonstrate autonomous vehicle reliability?" *Transportation Research Part A: Policy and Practice*, 94, 182-193.
   - Statistical challenges in AV validation
   - 11 billion miles problem

4. **Koopman, P., & Wagner, M.** (2016). "Challenges in Autonomous Vehicle Testing and Validation." *SAE International Journal of Transportation Safety*, 4(1), 15-24.
   - Comprehensive overview of AV testing challenges
   - Scenario-based approaches

5. **Tuncali, C. E., et al.** (2020). "Simulation-Based Adversarial Test Generation for Autonomous Vehicles." *ACM/IEEE International Conference on Cyber-Physical Systems*.
   - Adversarial testing methods
   - Automated scenario generation

6. **Riedmaier, S., et al.** (2020). "Survey on Scenario-Based Safety Assessment of Automated Vehicles." *IEEE Access*, 8, 87456-87477.
   - State-of-the-art scenario-based testing
   - Comprehensive taxonomy

7. **Peng, X., et al.** (2020). "A Survey on Software Testing for Machine Learning." *ACM Transactions on Software Engineering and Methodology*.
   - ML-specific testing challenges
   - Relevant for AV perception systems

### Papers - Simulation

8. **Dosovitskiy, A., et al.** (2017). "CARLA: An Open Urban Driving Simulator." *Conference on Robot Learning*.
   - Open-source AV simulator
   - Benchmarking platform

9. **Rong, G., et al.** (2020). "LGSVL Simulator: A High Fidelity Simulator for Autonomous Driving." *IEEE 23rd International Conference on Intelligent Transportation Systems*.
   - High-fidelity simulation
   - Sensor modeling

10. **Quiter, C. L., & Ernst, M. D.** (2021). "Simulation Fidelity in Autonomous Vehicle Safety Assessment." *IEEE Transactions on Intelligent Vehicles*.
    - Sim2Real gap analysis
    - Fidelity requirements

### Papers - Ethics

11. **Awad, E., et al.** (2018). "The Moral Machine Experiment." *Nature*, 563(7729), 59-64.
    - 40 million decisions from worldwide participants
    - Cultural variations in ethical preferences
    - Free data: https://www.moralmachine.net/

12. **Bonnefon, J. F., Shariff, A., & Rahwan, I.** (2016). "The Social Dilemma of Autonomous Vehicles." *Science*, 352(6293), 1573-1576.
    - Survey on AV ethics
    - Purchase paradox: want ethical AVs, buy selfish ones

13. **Lin, P.** (2016). "Why Ethics Matters for Autonomous Cars." In *Autonomous Driving* (pp. 69-85). Springer.
    - Philosophical analysis
    - Trolley problem adaptations

14. **Goodall, N. J.** (2014). "Ethical Decision Making During Automated Vehicle Crashes." *Transportation Research Record*, 2424(1), 58-65.
    - Framework for crash-imminent decisions
    - Legal and ethical considerations

15. **Nyholm, S., & Smids, J.** (2016). "The Ethics of Accident-Algorithms for Self-Driving Cars: An Applied Trolley Problem?" *Ethical Theory and Moral Practice*, 19(5), 1275-1289.
    - Critical analysis of trolley problem
    - Differences from traditional ethics

### Papers - Liability & Law

16. **Vladeck, D. C.** (2014). "Machines Without Principals: Liability Rules and Artificial Intelligence." *Washington Law Review*, 89, 117.
    - Legal liability frameworks for AI
    - Product liability vs. negligence

17. **Gurney, J. K.** (2013). "Sue My Car Not Me: Products Liability and Accidents Involving Autonomous Vehicles." *University of Illinois Journal of Law, Technology & Policy*, 2013, 247.
    - Product liability for AVs
    - Manufacturer vs. driver responsibility

18. **Schellekens, M.** (2018). "Self-Driving Cars and the Chilling Effect of Liability Law." *Computer Law & Security Review*, 31(4), 506-517.
    - How liability affects AV development
    - Regulatory recommendations

### Industry Reports & Standards

19. **UL 4600** - Standard for Safety for the Evaluation of Autonomous Products
    - First comprehensive AV safety standard
    - https://ul.org/UL4600

20. **NHTSA - Automated Vehicles for Safety** (2020)
    - Voluntary Safety Self-Assessment
    - https://www.nhtsa.gov/vehicle-safety/automated-vehicles-safety

21. **SAE J3016** - Taxonomy and Definitions for Automated Driving
    - Levels of automation (0-5)
    - ODD definitions

22. **BSI PAS 1883** - Operational Design Domain (ODD) Taxonomy
    - Systematic ODD specification
    - https://www.bsigroup.com/

### Tools & Simulators

23. **CARLA Simulator**
    - Open-source AV simulator
    - https://carla.org/
    - Python API, sensor simulation

24. **LGSVL Simulator**
    - Unity-based AV simulator
    - https://www.svlsimulator.com/
    - Digital twin capability

25. **SUMO** - Simulation of Urban MObility
    - Traffic flow simulation
    - https://www.eclipse.org/sumo/

26. **OpenScenario & OpenDrive**
    - Scenario description standards
    - https://www.asam.net/standards/detail/openscenario/
    - Industry-standard formats

### Datasets

27. **nuScenes**
    - Full sensor suite dataset
    - https://www.nuscenes.org/
    - Useful for testing perception

28. **Waymo Open Dataset**
    - Large-scale driving data
    - https://waymo.com/open/
    - Diverse scenarios

29. **KITTI**
    - Classic AV benchmark
    - http://www.cvlibs.net/datasets/kitti/
    - Stereo, lidar, GPS

### Ethics Resources

30. **Moral Machine Platform**
    - Interactive ethical dilemma explorer
    - https://www.moralmachine.net/
    - Research and education tool

31. **IEEE - Ethically Aligned Design**
    - Guide for autonomous systems
    - https://standards.ieee.org/industry-connections/ec/ead-v1/
    - Ethical considerations framework

### Courses

32. **Coursera - Self-Driving Cars Specialization** (University of Toronto)
    - Module 4: Safety assurance
    - Testing and validation

33. **edX - Autonomous Mobile Robots** (ETH Zurich)
    - Testing and simulation modules

34. **MIT OpenCourseWare - Ethics of Technology**
    - Philosophical foundations
    - Case studies including AVs